## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [14]:
from math import prod
import numpy as np

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def wl(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def wamp(time, threshold):
    return np.sum(np.abs(np.diff(time)) > threshold, axis=-1)

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [34]:
# carregando dados do "prepare.ipynb"

time = np.load("dataset/segmented_data_t.npy")
freq = np.load("dataset/segmented_data_f.npy")

tempo_participante1 = np.load("dataset/p1_tempo.npy")
frequencia_participante1 = np.load("dataset/p1_frequencia.npy")

tempo_participante2 = np.load("dataset/p2_tempo.npy")
frequencia_participante2 = np.load("dataset/p2_frequencia.npy")


# aplicando características participante 1

data_var_p_1 = var(tempo_participante1)
data_rms_p_1 = rms(tempo_participante1)
data_wl_p_1 = wl(tempo_participante1)
data_wamp_p_1 = wamp(tempo_participante1, 0.01)

data_fmd_p_1 = fmd(frequencia_participante1)
data_mmdf_p_1 = mmdf(frequencia_participante1)

# aplicando características participante 2

data_var_p_2 = var(tempo_participante2)
data_rms_p_2 = rms(tempo_participante2)
data_wl_p_2 = wl(tempo_participante2)
data_wamp_p_2 = wamp(tempo_participante2, 0.01)

data_fmd_p_2 = fmd(frequencia_participante2)
data_mmdf_p_2 = mmdf(frequencia_participante2)



In [38]:
# União do vetor de características inicial
features_participante_1 = np.array([data_wamp_p_1, data_var_p_1, data_rms_p_1, data_wl_p_1, data_fmd_p_1, data_mmdf_p_1]) 
print(features_participante_1.shape)

features_participante_2 = np.array([data_wamp_p_2, data_var_p_2, data_rms_p_2, data_wl_p_2, data_fmd_p_2, data_mmdf_p_2])
print(features_participante_2.shape)


(6, 28, 33, 2)
(6, 28, 33, 2)


In [39]:

# organização das dimensões participante 1
features_participante_1 = features_participante_1.transpose(1, 3, 0, 2)

# criar vetor de características definitivo participante 1
features_participante_1 = features_participante_1.reshape(features_participante_1.shape[0] * features_participante_1.shape[1],
                            features_participante_1.shape[2] * features_participante_1.shape[3])

# organização das dimensões participante 2
features_participante_2 = features_participante_2.transpose(1, 3, 0, 2)

# criar vetor de características definitivo participante 2
features_participante_2 = features_participante_2.reshape(features_participante_2.shape[0] * features_participante_2.shape[1],
                            features_participante_2.shape[2] * features_participante_2.shape[3])

features_participante_1.shape, features_participante_2.shape

((56, 198), (56, 198))

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [69]:
from sklearn.preprocessing import MinMaxScaler

X_norm_1 = MinMaxScaler().fit_transform(features_participante_1)
X_norm_2 = MinMaxScaler().fit_transform(features_participante_2)

X_norm_1.shape, X_norm_2.shape

((56, 198), (56, 198))

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

Criação do vetor de *labels*

In [70]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# criação do vetor de labels final
y = labels_num * int(features_participante_1.shape[0] / len(labels_num))
len(y)

56

*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [71]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

skf = StratifiedKFold(n_splits=4)
rfe = RFECV(SVC(kernel="linear"), min_features_to_select=1, step=2, cv=skf, scoring='accuracy')
X_full_1 = rfe.fit_transform(X_norm_1, y)
X_full_2 = rfe.fit_transform(X_norm_2, y)

X_full_1.shape, X_full_2.shape

((56, 1), (56, 26))

In [77]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


print("Aplicando SVM")

# Participante 1
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_full_1, y, test_size=0.2, random_state=7)

svm_model = SVC(kernel='linear').fit(X_train_1, y_train_1)
y_pred_1 = svm_model.predict(X_test_1)

print("Participante 1 - Accuracy: %.2f %%" % (accuracy_score(y_test_1, y_pred_1) * 100))

# Participante 2
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_full_2, y, test_size=0.2, random_state=2)

svm_model = SVC(kernel='linear').fit(X_train_2, y_train_2)
y_pred_2 = svm_model.predict(X_test_2)

print("Participante 2 - Accuracy: %.2f %%" % (accuracy_score(y_test_2, y_pred_2) * 100))

Aplicando SVM
Participante 1 - Accuracy: 41.67 %
Participante 2 - Accuracy: 58.33 %
